# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [48]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [49]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [50]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [51]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [57]:
def foldl2(f, x0, lst):
    return foldr(lambda v, k: lambda n: k(f(n, v)), lambda x: x, lst)(x0)
foldl2(lambda x, y: x + y, 'L', ['1', '2', '3'])


'L123'

In [53]:
def foldr2(f, x0, lst):
    return foldl(lambda k, v: lambda n: k(f(v, n)), lambda x: x, lst)(x0)
foldr2(lambda x, y: x + y, 'R', ['1', '2', '3'])

'123R'

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [54]:
# O(len(a))
import string
def check_inv(a, b): 
    assert len(a)>=len(b)
    b_ch = dict.fromkeys(string.ascii_letters, 0)
    for i in b:
        b_ch[i]+=1
        
    ch = dict.fromkeys(string.ascii_letters, 0)
    for i in a[:len(b)-1]:
        ch[i]+=1
    for i in  range(len(b)-1, len(a)):
        ch[a[i]]+=1
        if ch==b_ch:
            return True
        ch[a[i-len(b)+1]]-=1
    return False
check_inv( 'abcrotm', 'tro')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [55]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self): 
        if self.left == None and self.right == None:
            yield self.value
        if self.left:
            yield from self.left.__iter__()
        if self.right: 
            yield from self.right.__iter__()
            
    def __str__(self):
        r=[str(self.value)]
        if self.left:
            r.append(self.left.__str__())     
        if self.right != None:
            r.append(self.right.__str__())   
        return ' '.join(r)
    
    def __repr__(self):
        if self.left == None and self.right == None:
            return f'Tree({self.value})'
        if self.right == None:
            return f'Tree({self.value} , {self.left.__repr__()} , None)'
        if self.left == None:
            return f'Tree({self.value},None, {self.right.__repr__()})'
        
        return f'Tree({self.value},{self.left.__repr__()}, {self.right.__repr__()})'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))


print(list(tree)) #== [3, 4, 2]
print(tree)
tree
#tree

[3, 4, 2]
0 1 3 4 2


Tree(0,Tree(1,Tree(3), Tree(4)), Tree(2))

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [56]:
oper ={'*' :lambda a,b: a*b, '/' :lambda a,b: a/b, '+': lambda a,b: a+b,'-' :lambda a,b: b-a}
prior = {1: ['+', '-'], 2: ['*', '/'], 0:['(', ')']}
 
def priority(val):
    for k, v in prior.items():
        if val in v:
            return k
    return -1

def splt(st):
    s = []
    for i in st.split():
        if len(i)>1:
            if i[0] == '(':
                s.append(i[0])
                s.append(i[1:])
            elif i[-1] == ')':
                s.append(i[:-1])
                s.append(i[-1])
            else: s.append(i)
        else: s.append(i)
    return s

def calc(expr):
    expr = splt(expr)
    result = []
    stack = []
    for el in expr:
        if el not in '+-*/()':
            result.append(int(el))
        else:
            if el == '(': 
                stack.append(el)
                continue
            elif el == ')': 
                while True:
                    result.append(oper[stack.pop()](result.pop(),result.pop()))
                    if stack[-1]=='(':
                        stack.pop()
                        break
            else:    
                if len(stack) == 0: last = None 
                else: last = stack[-1]
                while priority(last) >= priority(el):
                    result.append(oper[stack.pop()](result.pop(),result.pop()))
                    if len(stack) == 0: last = None 
                    else: last = stack[-1]
                stack.append(el)
    while True:
        if len(stack)>0:
            result.append(oper[stack.pop()](result.pop(),result.pop()))
        else: break 
    return result[0]

    
calc('2 * (15 - 3 * 4) - 2') == 4

True